In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import numpy as np 
import pandas as pd 
import os 
import kagglehub
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.applications.mobilenet_v2 import MobileNetV2
import tensorflow as tf
print(tf.__version__)

2025-03-13 00:33:31.886880: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-13 00:33:31.941608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-13 00:33:31.981725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-13 00:33:31.993663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 00:33:32.024583: I tensorflow/core/platform/cpu_feature_guar

2.17.0


In [2]:
dataset_path = kagglehub.dataset_download("paramaggarwal/fashion-product-images-small")
print("Path to dataset files:", dataset_path)

Path to dataset files: /nfs/home/mjc3869/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-small/versions/1


In [3]:
dataset_path = '/nfs/home/mjc3869/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-small/versions/1'

In [4]:
# Define dataset paths
image_folder = os.path.join(dataset_path, "images")
metadata_path = os.path.join(dataset_path, "styles.csv")

# Step 2: Load metadata
df = pd.read_csv(metadata_path, on_bad_lines='skip')

# Filter dataset (only use images present in folder)
df["image_path"] = df["id"].astype(str) + ".jpg"
df = df[df["image_path"].isin(os.listdir(image_folder))]

# Select a subset of classes for simplicity
df = df[df['masterCategory'].isin(['Apparel', 'Footwear', 'Accessories'])]

# Encode labels
label_map = {label: idx for idx, label in enumerate(df["masterCategory"].unique())}
df["category_id"] = df["masterCategory"].map(label_map)

# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["category_id"], random_state=42)

In [5]:
from keras_preprocessing.image import ImageDataGenerator

image_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col="image_path", y_col="masterCategory",
    target_size=image_size, batch_size=batch_size, class_mode="categorical", subset="training"
)

val_generator = datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col="image_path", y_col="masterCategory",
    target_size=image_size, batch_size=batch_size, class_mode="categorical", subset="validation"
)

Found 26807 validated image filenames belonging to 3 classes.
Found 6701 validated image filenames belonging to 3 classes.


In [16]:
from tensorflow.keras import layers, models
def cnn_model(input_shape=(128, 128, 3), num_classes=3):
    model = models.Sequential([
        # Convolutional layers
        layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2,2)),
        
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),

        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Example usage
model = cnn_model(input_shape=(128, 128, 3), num_classes=3)
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1024)           │    33,555,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │         3,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,651,779 (128.37 MB)

 Trainable params: 33,651,779 (128.37 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10


I0000 00:00:1741827241.980786 1922858 service.cc:146] XLA service 0x7f03140050e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741827241.980863 1922858 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741827241.980879 1922858 service.cc:154]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741827241.980883 1922858 service.cc:154]   StreamExecutor device (2): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741827241.980887 1922858 service.cc:154]   StreamExecutor device (3): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741827241.980891 1922858 service.cc:154]   StreamExecutor device (4): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741827241.980897 1922858 service.cc:154]   StreamExecutor device (5): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1741827241.9809

  3/838 ━━━━━━━━━━━━━━━━━━━━ 1:02 75ms/step - accuracy: 0.5069 - loss: 8.3318 

I0000 00:00:1741827246.354164 1922858 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


838/838 ━━━━━━━━━━━━━━━━━━━━ 110s 123ms/step - accuracy: 0.8994 - loss: 0.4786 - val_accuracy: 0.9725 - val_loss: 0.1007
Epoch 2/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 119s 100ms/step - accuracy: 0.9755 - loss: 0.0760 - val_accuracy: 0.9828 - val_loss: 0.0495
Epoch 3/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 132s 88ms/step - accuracy: 0.9875 - loss: 0.0414 - val_accuracy: 0.9870 - val_loss: 0.0442
Epoch 4/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 85s 92ms/step - accuracy: 0.9908 - loss: 0.0295 - val_accuracy: 0.9885 - val_loss: 0.0439
Epoch 5/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 73s 82ms/step - accuracy: 0.9942 - loss: 0.0167 - val_accuracy: 0.9884 - val_loss: 0.0446
Epoch 6/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 90s 91ms/step - accuracy: 0.9947 - loss: 0.0167 - val_accuracy: 0.9885 - val_loss: 0.0582
Epoch 7/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 78s 86ms/step - accuracy: 0.9958 - loss: 0.0124 - val_accuracy: 0.9893 - val_loss: 0.0554
Epoch 8/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 87s 93ms/step - accuracy: 0.9992 - loss: 0.0030 - val_ac

In [18]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col="image_path", y_col="masterCategory",
    target_size=image_size, batch_size=batch_size, class_mode="categorical", shuffle=False
)

loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy:.2f}")

# Save Model
model.save("fashion_classifier_resnet.h5")

Found 8377 validated image filenames belonging to 3 classes.


/nfs/home/mjc3869/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


262/262 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.9863 - loss: 0.0754


Test Accuracy: 0.99


In [20]:
# EfficientNet
from tensorflow.keras.applications import EfficientNetB0
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation="relu"),
    layers.Dense(len(label_map), activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train Model
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 151s 138ms/step - accuracy: 0.4878 - loss: 1.0684 - val_accuracy: 0.5193 - val_loss: 1.0234
Epoch 2/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 81s 87ms/step - accuracy: 0.5039 - loss: 1.0353 - val_accuracy: 0.5193 - val_loss: 1.0198
Epoch 3/10
838/838 ━━━━━━━━━━━━━━━━━━━━ 72s 86ms/step - accuracy: 0.5096 - loss: 1.0279 - val_accuracy: 0.5193 - val_loss: 1.0189
Epoch 4/10
257/838 ━━━━━━━━━━━━━━━━━━━━ 40s 70ms/step - accuracy: 0.5082 - loss: 1.0280

KeyboardInterrupt: 